In [ ]:
# サンプルデータセットをダウンロード
%cd /content
!git clone https://github.com/uedake/yolo_sample
%cd yolo_sample
!git pull
%cd /content

In [ ]:
# データセット中のラベルの集計を表示
!python yolo_sample/label_stat.py

In [ ]:
# 学習用のライブラリをインストール
%pip install -q ultralytics

!yolo settings tensorboard=True
%load_ext tensorboard

In [ ]:
# 学習の実行
import ultralytics
from ultralytics import YOLO

ultralytics.checks()
model = YOLO('yolo11n.pt') # load a pretrained model
model.train(data='yolo_sample/dataset/sample1/custom.yaml', exist_ok=True, epochs=60)

In [ ]:
# 学習したモデルを使用した推論の実行
from glob import glob
import logging

from ultralytics.utils import LOGGER
import matplotlib.pyplot as plt

def show_all_images(img_list,ncols=4):
  r=0
  c=0
  for img in img_list:
    if c==0:
      fig, axes = plt.subplots(1, ncols, figsize=(8*ncols,6))
    axes[c].imshow(img)
    axes[c].axis('off')
    c+=1
    if c==ncols:
      c=0
      r+=1
  plt.tight_layout()
  plt.show()

LOGGER.setLevel(logging.ERROR)
results = model(glob("yolo_sample/dataset/sample1/images/val/*.jpg"))
LOGGER.setLevel(logging.INFO)

show_all_images([r.plot()[...,::-1] for r in results])

In [ ]:
# 参考：学習時の評価で使用された画像の表示
import matplotlib.image as mpimg


def show_2_images(img1,img2):
  fig, axes = plt.subplots(1, 2, figsize=(20, 12))  # 横に2枚、サイズ指定

  axes[0].imshow(img1)
  axes[0].axis('off')
  axes[0].set_title("original")
  axes[1].imshow(img2)
  axes[1].axis('off')
  axes[1].set_title("predict")
  plt.tight_layout()
  plt.show()

for path in glob("runs/detect/train/val_batch*_labels.jpg"):
  show_2_images(mpimg.imread(path),mpimg.imread(path[:-len("labels.jpg")]+"pred.jpg"))

In [ ]:
# 参考：学習結果の確認ツールの起動
%tensorboard --logdir runs

In [ ]:
# モデルの出力
model.export(format="onnx")
!ls runs/detect/train/weights -l

In [ ]:
model_best = YOLO('runs/detect/train/weights/best.onnx') # load the trained model
LOGGER.setLevel(logging.ERROR)
results_best = [model_best(path) for path in glob("yolo_sample/dataset/sample1/images/val/*.jpg")]
LOGGER.setLevel(logging.INFO)

show_all_images([r[0].plot()[...,::-1] for r in results_best])

In [ ]:
#onnxモデルの推論時間の計測
%%timeit -r 10
LOGGER.setLevel(logging.ERROR)
_ = model_best(glob("yolo_sample/dataset/sample1/images/val/*.jpg")[0])
LOGGER.setLevel(logging.INFO)

In [ ]:
# 学習済みモデルのダウンロード
from google.colab import files

files.download('runs/detect/train/weights/best.onnx')